# Setting up data for yield curve arbitrage #

### Reading and parsing daily data. Month-end dates are chosen at the end ###

In [1]:
# Importing necessary libraries
import pandas as pd

In [2]:
# Reading in daily USD data
xls_file = pd.ExcelFile('usd data.xlsx')

# Parsing daily data: ICE LIBOR rates / ICE swap rates / Tullett Prebon OIS rates
ice_libor_daily = xls_file.parse('ICE_LIBOR')
ice_swap_daily = xls_file.parse('ICE_ISDAFIX_SWAP') 
tullett_ois = xls_file.parse('TULLETT_OIS')

### Preparing daily Tullett Prebon OIS data ###

In [3]:
# Dropping columns with all nans, setting initial column names and dropping some unnecessary rows
tullett_ois = tullett_ois.dropna(axis=1, how='all')
tullett_ois.columns = tullett_ois.iloc[0]
tullett_ois = tullett_ois.drop([0,1]).iloc[0:].reset_index(drop=True)

# Setting up the columns correctly
def helper_col(x):
    if isinstance(x, str):
        return x.replace("=TTKL", "")
tullett_ois.columns = ['date'] + list(map(helper_col, tullett_ois.columns.astype("str")))[1:]
tullett_ois = tullett_ois.drop(['nan'], axis=1)

# Reversing the order of rows and dropping rows with nan-values
tullett_ois = tullett_ois.reindex(index=tullett_ois.index[::-1]).reset_index(drop=True)
tullett_ois = tullett_ois.dropna(how='any').reset_index(drop=True)

# Setting up the date column to datetime, dividing values by 100 to get the percentages in correct form and printing the dataframe
tullett_ois['date'] = pd.to_datetime(tullett_ois['date'])
tullett_ois.loc[:,'USDSWOIS':] = tullett_ois.loc[:,'USDSWOIS':].div(100)
tullett_ois

,date,USDSWOIS,USD2WOIS,USD3WOIS,USD1MOIS,USD2MOIS,USD3MOIS,USD4MOIS,USD5MOIS,USD6MOIS,...,USD4YOIS,USD51MOIS,USD54MOIS,USD57MOIS,USD5YOIS,USD6YOIS,USD7YOIS,USD8YOIS,USD9YOIS,USD10YOIS
0,2011-09-08,0.00094,0.00087,0.00084,0.00082,0.0008,0.00079,0.00079,0.00078,0.00078,...,0.00475,0.00629,0.00705,0.00768,0.0085,0.01116,0.01351,0.01549,0.01713,0.01851
1,2011-09-09,0.00106,0.00092,0.00087,0.00084,0.00081,0.0008,0.00078,0.00078,0.00079,...,0.0048,0.00664,0.00739,0.00806,0.00883,0.0115,0.0138,0.01572,0.01733,0.01867
2,2011-09-12,0.00103,0.00093,0.00089,0.00087,0.00085,0.00084,0.00082,0.00082,0.00082,...,0.00472,0.00634,0.00705,0.00772,0.00845,0.01104,0.01327,0.01513,0.01668,0.01796
3,2011-09-13,0.00108,0.00093,0.00088,0.00084,0.00081,0.0008,0.0008,0.00081,0.00082,...,0.00435,0.00579,0.00646,0.00703,0.00772,0.01011,0.01223,0.01397,0.01538,0.01656
4,2011-09-14,0.0009,0.00081,0.00078,0.00077,0.00075,0.00074,0.00071,0.00071,0.00071,...,0.00505,0.00617,0.00696,0.00747,0.00825,0.0107,0.01287,0.01465,0.01612,0.01734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,2021-12-27,0.00073,0.00076,0.00077,0.00079,0.00084,0.00106,0.00142,0.00179,0.00213,...,0.01056,0.01075,0.0109,0.01106,0.01115,0.01158,0.01196,0.01238,0.01258,0.0129
2382,2021-12-28,0.00077,0.0008,0.00081,0.00083,0.00089,0.00113,0.00149,0.00186,0.00222,...,0.01054,0.01074,0.0109,0.01106,0.01114,0.01158,0.01196,0.01243,0.01262,0.01293
2383,2021-12-29,0.00073,0.00075,0.00076,0.00078,0.00084,0.00112,0.00146,0.00184,0.0022,...,0.01084,0.01105,0.01125,0.01141,0.01153,0.01207,0.01252,0.01291,0.01328,0.01361
2384,2021-12-30,0.00081,0.00081,0.00081,0.00083,0.00086,0.00116,0.00146,0.00182,0.0022,...,0.01058,0.01078,0.01095,0.01112,0.01124,0.01173,0.01213,0.01249,0.01286,0.01318


### Preparing daily ICE swap data ###

In [4]:
# Setting up the columns correctly
ice_swap_daily.columns = ['date', 'USD1YS', 'USD2YS', 'USD3YS', 'USD4YS', 'USD5YS', 'USD6YS', 'USD7YS', 'USD8YS', 'USD9YS', 'USD10YS']
ice_swap_daily['date'] = pd.to_datetime(ice_swap_daily['date'])

# Dropping rows containing nan-values
ice_swap_daily = ice_swap_daily.dropna(how='any').reset_index(drop=True)

# Dividing values by 100 to get the percentages in correct form
ice_swap_daily.loc[:, 'USD1YS':] = ice_swap_daily.loc[:, 'USD1YS':].div(100)
ice_swap_daily

,date,USD1YS,USD2YS,USD3YS,USD4YS,USD5YS,USD6YS,USD7YS,USD8YS,USD9YS,USD10YS
0,2007-01-19,0.05397,0.05265,0.05200,0.05184,0.05190,0.05202,0.05217,0.05235,0.05253,0.05268
1,2007-01-22,0.05395,0.05249,0.05179,0.05163,0.05162,0.05173,0.05188,0.05204,0.05221,0.05240
2,2007-01-23,0.05408,0.05271,0.05195,0.05179,0.05186,0.05198,0.05210,0.05227,0.05244,0.05264
3,2007-01-24,0.05419,0.05289,0.05224,0.05213,0.05219,0.05232,0.05250,0.05266,0.05284,0.05302
4,2007-01-25,0.05429,0.05309,0.05247,0.05237,0.05245,0.05258,0.05276,0.05292,0.05310,0.05331
...,...,...,...,...,...,...,...,...,...,...,...
3910,2022-01-14,0.00668,0.01142,0.01402,0.01529,0.01608,0.01668,0.01716,0.01752,0.01780,0.01808
3911,2022-01-17,0.00668,0.01142,0.01402,0.01529,0.01608,0.01668,0.01716,0.01752,0.01780,0.01808
3912,2022-01-18,0.00738,0.01222,0.01488,0.01625,0.01709,0.01772,0.01820,0.01855,0.01882,0.01910
3913,2022-01-19,0.00722,0.01204,0.01478,0.01622,0.01711,0.01774,0.01822,0.01855,0.01881,0.01906


### Preparing daily ICE LIBOR data ###

In [5]:
# Setting up the columns correctly
ice_libor_daily.columns = ['date', 'USDONL', 'USD1WL', 'USD1ML', 'USD2ML', 'USD3ML', 'USD6ML', 'USD12ML']
ice_libor_daily['date'] = pd.to_datetime(ice_libor_daily['date'])

# Dropping rows containing nan-values
ice_libor_daily = ice_libor_daily.dropna(how='any').reset_index(drop=True)

# Dividing values by 100 to get the percentages in correct form
ice_libor_daily.loc[:,'USDONL':] = ice_libor_daily.loc[:,'USDONL':].div(100)
ice_libor_daily

,date,USDONL,USD1WL,USD1ML,USD2ML,USD3ML,USD6ML,USD12ML
0,2007-01-19,0.052888,0.053038,0.053200,0.053444,0.053600,0.053900,0.053869
1,2007-01-22,0.052881,0.053006,0.053200,0.053437,0.053600,0.053944,0.054069
2,2007-01-23,0.052906,0.053044,0.053200,0.053437,0.053600,0.053906,0.054025
3,2007-01-24,0.052856,0.053019,0.053200,0.053450,0.053600,0.053987,0.054200
4,2007-01-25,0.052850,0.053050,0.053200,0.053444,0.053600,0.053987,0.054169
...,...,...,...,...,...,...,...,...
3910,2022-01-14,0.000740,0.000764,0.001033,0.001525,0.002413,0.003950,0.007257
3911,2022-01-17,0.000740,0.000764,0.001030,0.001525,0.002480,0.004181,0.007667
3912,2022-01-18,0.000703,0.000764,0.001037,0.001525,0.002540,0.004299,0.007870
3913,2022-01-19,0.000781,0.000764,0.001091,0.001525,0.002551,0.004471,0.008036


### Merging the data by date as 'data1_merge'-dataframe and choosing the month-end dates. Merging the datasets by date gives data from 09/2011 forwards. The overall data consists of: ### 
### Daily ICE LIBOR data: overnight, 1 week, 1 month, 2 months, 3 months, 6 months, 12 months ### 
### Daily ICE swap data: 1 year - 10 years ### 
### Daily Tullett Prebon OIS data: 1 week, 2 weeks, 3 weeks, 1-12 months, 13 months+ ###

In [6]:
# Merging first the daily ICE swap data with daily ICE LIBOR data
data1_merge = ice_swap_daily.merge(ice_libor_daily, how='inner', on='date')

# Merging the ICE swap-LIBOR dataframe to the dataframe of Tullett OIS data
data1_merge = data1_merge.merge(tullett_ois, how='inner', on='date')

# Dropping rows with all zero values
data1_merge = data1_merge[(data1_merge != 0).all(1)].reset_index(drop = True)

# Choosing month-end dates
data1_merge = data1_merge.groupby([data1_merge['date'].apply(lambda x: x.year), data1_merge['date'].apply(lambda x: x.month)]).apply(lambda x: x.sort_values(by='date', axis=0).tail(1)).reset_index(drop=True)

### Setting up separate dataframes for separate data in the merged dataframe ###

In [7]:
# Setting up the swap rates

# Pre-trading month-end dates for swap rates. This expanded dataset is needed for modelling the swap rates
pre_trade_swaps = ice_swap_daily.loc[0:1203,:]
pre_trade_swaps = pre_trade_swaps.groupby([pre_trade_swaps['date'].apply(lambda x: x.year), pre_trade_swaps['date'].apply(lambda x: x.month)]).apply(lambda x: x.sort_values(by='date', axis=0).tail(1)).reset_index(drop=True)

# Swaps of trading dates
data1_swap_trade = data1_merge.loc[:,'date':'USD10YS']

# Combine pre-trading swap rates to the trading swap rates to have the full dataset for modelling
data1_swap_model = pd.concat([pre_trade_swaps, data1_swap_trade], ignore_index=True)

In [8]:
# Setting up the LIBOR rates

# Full LIBOR dataset
data1_libor = data1_merge[['date'] + list(data1_merge.loc[:, 'USDONL':'USD12ML'])]

# 3M LIBOR rates
data1_libor_3M_original = data1_merge[['date'] + ['USD3ML']]

In [9]:
# Setting up the OIS rates
data1_ois = data1_merge[['date'] + list(data1_merge.loc[:, 'USDSWOIS':'USD10YOIS'])]

In [10]:
data1_ois.columns

Index(['date', 'USDSWOIS', 'USD2WOIS', 'USD3WOIS', 'USD1MOIS', 'USD2MOIS',
       'USD3MOIS', 'USD4MOIS', 'USD5MOIS', 'USD6MOIS', 'USD7MOIS', 'USD8MOIS',
       'USD9MOIS', 'USD10MOIS', 'USD11MOIS', 'USD1YOIS', 'USD13MOIS',
       'USD14MOIS', 'USD15MOIS', 'USD16MOIS', 'USD17MOIS', 'USD18MOIS',
       'USD19MOIS', 'USD20MOIS', 'USD21MOIS', 'USD22MOIS', 'USD23MOIS',
       'USD2YOIS', 'USD27MOIS', 'USD30MOIS', 'USD33MOIS', 'USD3YOIS',
       'USD39MOIS', 'USD42MOIS', 'USD45MOIS', 'USD4YOIS', 'USD51MOIS',
       'USD54MOIS', 'USD57MOIS', 'USD5YOIS', 'USD6YOIS', 'USD7YOIS',
       'USD8YOIS', 'USD9YOIS', 'USD10YOIS'],
      dtype='object')

In [11]:
data1_merge

,date,USD1YS,USD2YS,USD3YS,USD4YS,USD5YS,USD6YS,USD7YS,USD8YS,USD9YS,...,USD4YOIS,USD51MOIS,USD54MOIS,USD57MOIS,USD5YOIS,USD6YOIS,USD7YOIS,USD8YOIS,USD9YOIS,USD10YOIS
0,2011-09-30,0.00530,0.00574,0.00737,0.00980,0.01254,0.01494,0.01692,0.01855,0.01989,...,0.00693,0.00711,0.00803,0.00869,0.00968,0.01275,0.01525,0.01732,0.01906,0.02048
1,2011-10-31,0.00523,0.00563,0.00731,0.01008,0.01321,0.01615,0.01864,0.02063,0.02224,...,0.00508,0.00592,0.00658,0.00728,0.00804,0.01075,0.01296,0.01483,0.01644,0.0178
2,2011-11-30,0.00620,0.00656,0.00798,0.01051,0.01329,0.01590,0.01801,0.01980,0.02119,...,0.00507,0.0057,0.0063,0.00703,0.00767,0.01024,0.01239,0.0142,0.01575,0.01707
3,2011-12-30,0.00682,0.00736,0.00837,0.01030,0.01249,0.01465,0.01649,0.01806,0.01932,...,0.00474,0.00545,0.00622,0.00676,0.00762,0.0103,0.01257,0.01452,0.01619,0.01761
4,2012-01-31,0.00496,0.00517,0.00599,0.00780,0.01003,0.01258,0.01472,0.01656,0.01809,...,0.00538,0.0061,0.00629,0.00733,0.00761,0.01022,0.01246,0.0144,0.01605,0.01748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2021-08-31,0.00154,0.00301,0.00528,0.00718,0.00868,0.00996,0.01102,0.01187,0.01257,...,0.00552,0.00585,0.00619,0.00652,0.00686,0.00805,0.00903,0.00985,0.0105,0.01111
120,2021-09-30,0.00176,0.00395,0.00669,0.00903,0.01083,0.01224,0.01335,0.01420,0.01489,...,0.00695,0.00737,0.00777,0.00814,0.0085,0.00974,0.01074,0.01151,0.0121,0.01265
121,2021-10-29,0.00314,0.00702,0.00983,0.01140,0.01246,0.01340,0.01417,0.01477,0.01525,...,0.00927,0.00948,0.00973,0.00996,0.01023,0.01106,0.01185,0.01238,0.01283,0.01325
122,2021-11-29,0.00346,0.00742,0.01034,0.01191,0.01291,0.01376,0.01446,0.01496,0.01534,...,0.00951,0.00979,0.01004,0.01025,0.01044,0.01126,0.0119,0.01234,0.01274,0.01306


### Statistics for calibration ###

In [12]:
from statistics import mean
from statistics import stdev

***USD overnight LIBOR***

In [13]:
ice_libor_daily.iloc[1312,:]

date       2012-01-31 00:00:00
USDONL                0.001395
USD1WL                0.001965
USD1ML                0.002647
USD2ML                0.003945
USD3ML                0.005424
USD6ML                0.007783
USD12ML               0.010957
Name: 1312, dtype: object

In [14]:
#mean(ice_libor_daily.loc[0:1312,'USDONL'])
#mean(ice_libor_daily['USDONL'])
max(ice_libor_daily['USDONL'])

0.06875

In [15]:
#stdev(ice_libor_daily.loc[0:1312,'USDONL'])
stdev(ice_libor_daily['USDONL'])

0.013801554235373253

***USD 1 week LIBOR***

In [16]:
#mean(ice_libor_daily.loc[0:1312,'USDONL'])
max(ice_libor_daily['USD1WL'])

0.0600625

In [17]:
#stdev(ice_libor_daily.loc[0:1312,'USDONL'])
#stdev(ice_libor_daily['USD1WL'])
stdev(ice_libor_daily.loc[2000:,'USDONL'])

0.008200354104913222

***USD 1 week OIS***

In [18]:
tullett_ois.iloc[103,:]

date         2012-01-31 00:00:00
USDSWOIS                 0.00093
USD2WOIS                 0.00093
USD3WOIS                 0.00093
USD1MOIS                 0.00093
USD2MOIS                 0.00095
USD3MOIS                 0.00096
USD4MOIS                 0.00096
USD5MOIS                 0.00097
USD6MOIS                 0.00098
USD7MOIS                 0.00098
USD8MOIS                 0.00099
USD9MOIS                   0.001
USD10MOIS                0.00101
USD11MOIS                0.00102
USD1YOIS                 0.00103
USD13MOIS                0.00117
USD14MOIS                0.00119
USD15MOIS                0.00121
USD16MOIS                0.00123
USD17MOIS                0.00125
USD18MOIS                0.00128
USD19MOIS                 0.0013
USD20MOIS                0.00133
USD21MOIS                0.00136
USD22MOIS                0.00138
USD23MOIS                0.00141
USD2YOIS                 0.00146
USD27MOIS                0.00213
USD30MOIS                0.00241
USD33MOIS 

In [19]:
#mean(tullett_ois.loc[0:103,'USDSWOIS'])
max(tullett_ois['USDSWOIS'])

0.02451

In [20]:
#stdev(tullett_ois.loc[0:103,'USDSWOIS'])
stdev(tullett_ois['USDSWOIS'])

0.007595594266502242

In [22]:
data1_libor.to_excel('liibordata.xlsx')

In [25]:
data1_ois.to_excel('ooisdata.xlsx')